In [1]:
import glob
import cv2
#import pillow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
############################
#### CAMERA CALIBRATION ####
############################

## Initialization ##

nx = 9 # no. of inside corners in x
ny = 6 # no. of inside corners in y

cal_img_list = []
undistorted_img = []
warped_img = []

# Make a list of calibration images
raw_img_dir = './camera_cal/calibration*.jpg'
for fname in glob.glob(raw_img_dir):
    image = cv2.imread(fname)
    cal_img_list.append(image)

cal_img_list = np.array(cal_img_list)    
    
# Array to store object points and image points from all the images
object_points = [] #3D points in real world space
image_points = [] #2D points in image plane

# prespare object points
objpts = np.zeros((ny*nx,3),np.float32) 
objpts[:,:2] = np.mgrid[0:6,0:9].T.reshape(-1,2) #list down x & y co-ordinates
#print(objpts[:,:2])

## Find Image & Object Points ##

for img in cal_img_list:
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (ny, nx), None)
    if ret == True:
        image_points.append(corners)
        object_points.append(objpts)
        undistorted_img.append(img)
    else:
        #plt.imshow(img)
        #plt.show()
        #print('Corners not found!')
        pass
        

print('No. of Images whose corners were found')
print(len(undistorted_img))

# Calibrate Camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)

No. of Images whose corners were found
17


In [3]:
## Test Undistortion ##

def plot(img1, img2, label1, label2, fname):
    f, (p1, p2) = plt.subplots(1,2, figsize=(20,12))
    p1.imshow(img1)
    p1.set_title(label1, fontsize=25)
    p2.imshow(img2, cmap='gray')
    p2.set_title(label2, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
#     f.savefig('output_images/'+fname+'.png') #save last pic for report

for a in range(0,len(undistorted_img)):
    test_img1 = undistorted_img[a]
    undst1 = cv2.undistort(test_img1, mtx, dist, None, mtx)
    #plot(test_img1, undst1, 'Original', 'Undistorted')

#f.savefig('output_images/Test_Undistorted.png') #save last pic for report

In [4]:
## Perspective Transform ##

def corners_warp(img, nx, ny):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray,(nx,ny),None)
    img_size = gray.shape
    offset = 100
    if ret == True:
        cv2.drawChessboardCorners(img, (nx,ny),corners,ret)
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        dest = np.float32([[offset,offset],[img_size[1]-offset,offset],
                           [img_size[1]-offset,img_size[0]-offset],[offset,img_size[0]-offset]])
        M = cv2.getPerspectiveTransform(src,dest)
        warped = cv2.warpPerspective(img, M, (img_size[1], img_size[0]), flags=cv2.INTER_LINEAR)
        return warped, M

In [5]:
## Test Undistortion & Warping ##

for a in range(0, len(undistorted_img)):
    test_img2 = undistorted_img[a]
    undst2 = cv2.undistort(test_img2,mtx,dist,None,mtx)
    if a is not 10:
        top_down, perspective_M = corners_warp(undst2, nx, ny)
        #plot(test_img2, top_down, 'Original', 'Undistorted')

#f.savefig('output_images/Test_Undistortion_Warping.png') #save last pic for report

In [6]:
###############################################################################################################################

In [7]:
######################
#### LANE FINDING ####
######################

In [8]:
## Gradient & Color Transform ##

def abs_sobel_thresh(img, orient, thresh):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray,cv2.CV_64F,1,0)
    elif orient == 'y':
        sobel = cv2.Sobel(gray,cv2.CV_64F,0,1)
    else:
        print("Select Appropriate Orientation!")
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    Sxbinary = np.zeros_like(scaled_sobel)
    Sxbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel<= thresh[1])]=1
    return Sxbinary
    
def mag_threshold(img, sobel_kernel, mag_thresh):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel )
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    abs_sobelxy = np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel > mag_thresh[0]) & (scaled_sobel < mag_thresh[1])]=1
    return binary_output

def dir_threshold(img, sobel_kernel, dir_thresh):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    abs_sobelx=np.absolute(sobelx)
    abs_sobely=np.absolute(sobely)
    dir_sobel=np.arctan2(abs_sobely,abs_sobelx)
    binary_output=np.zeros_like(dir_sobel)
    binary_output[(dir_sobel>=dir_thresh[0]) & (dir_sobel<=dir_thresh[1])]=1
    return binary_output

def hls_select(img, threshS):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[(S>threshS[0]) & (S<=threshS[1])] = 1
    return binary_output

def lab_bthresh(img, thresh):
    # 1) Convert to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    lab_b = lab[:,:,2]
    # don't normalize if there are no yellows in the image
    if np.max(lab_b) > 175:
        lab_b = lab_b*(255/np.max(lab_b))
    # 2) Apply a threshold to the L channel
    binary_output = np.zeros_like(lab_b)
    binary_output[((lab_b > thresh[0]) & (lab_b <= thresh[1]))] = 1
    # 3) Return a binary image of threshold result
    return binary_output

# Initialization #

# test_img = cv2.imread('./test_images/test5.jpg')
# undst = cv2.undistort(test_img, mtx, dist, None, mtx)
# # plot(test_img, undst, 'Original', 'Undistorted', 'Undistorted Image')

# ksize = 15

# gradx = abs_sobel_thresh(undst, orient='x', thresh=(20,255))
# # plot(test_img, gradx, 'Original', 'Gradient X Threshold', 'Gradient X Threshold')

# grady = abs_sobel_thresh(undst, orient='y', thresh=(40,255))
# # plot(test_img, grady, 'Original', 'Gradient Y Threshold', 'Gradient Y Threshold')

# mag_binary = mag_threshold(undst, sobel_kernel=ksize, mag_thresh=(60,255))
# # plot(test_img, mag_binary, 'Original', 'Gradient Mag_Threshold', 'Gradient Mag_Threshold')

# dir_binary = dir_threshold(undst, sobel_kernel=ksize, dir_thresh=(0.5,1.3))
# # plot(test_img, dir_binary, 'Original', 'Gradient Dir_Threshold', 'Gradient Dir_Threshold')

# combined_gradient = np.zeros_like(dir_binary)
# combined_gradient[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
# # plot(test_img, combined_gradient, 'Original', 'Final Grayscale', 'Final Grayscale')

# SL_Filter = hls_select(test_img, threshS=(160,255))
# # plot(test_img, SL_Filter, 'Original', 'HLS SL Color Filter', 'HLS SL Color Filter')

# B_Filter = lab_bthresh(test_img, thresh=(0,100))
# # plot(test_img, B_Filter, 'Original', 'LAB B Color Filter', 'LAB B Color Filter')

# color_filters = np.zeros_like(SL_Filter)
# color_filters[(SL_Filter == 1) | (B_Filter == 1)] = 1
# # plot(test_img, color_filters, 'Original', 'Combined Color Filters', 'Combined Color Filters')

# Final_Grad_Image = np.zeros_like(color_filters)
# Final_Grad_Image[(color_filters == 1) | (combined_gradient == 1)] = 1
# # plot(test_img, Final_Grad_Image, 'Original', 'Final Gradient Image', 'Final Gradient Image')

# # Stack each channel to view their individual contributions in green and blue respectively
# # This returns a stack of the two binary images, whose components you can see as different colors
# color_binary = np.zeros_like(combined_gradient)
# color_binary = np.uint8(np.dstack(( np.zeros_like(combined_gradient), combined_gradient, color_filters)) * 255)
# # plot(color_binary, Final_Grad_Image, 'Stacked Thresholds', 'Final Image', 'Stacked Thresholds')

In [9]:
## Warping the Lanes ##

def plot(img1, img2, label1, label2, fname):
    f, (p1, p2) = plt.subplots(1,2, figsize=(20,12))
    p1.imshow(img1, cmap='gray')
    p1.set_title(label1, fontsize=25)
    p2.imshow(img2, cmap='gray')
    p2.set_title(label2, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
#     f.savefig('output_images/'+fname+'.png') #save last pic for report

def warping(img):
    img_size = img.shape
    src = np.float32(np.array([[200,720],[600,450],[700,450],[1100,720]]))
    dest = np.float32(np.array([[350,720],[350,0],[950,0],[950,720]]))
    pt1 = np.float32(np.array([[0,720],[0,0],[200,0],[200,720]]))
    pt2 = np.float32(np.array([[1100,720],[1100,0],[1280,0],[1280,720]]))
    M = cv2.getPerspectiveTransform(src,dest)
    Minv = cv2.getPerspectiveTransform(dest, src)
    warped = cv2.warpPerspective(img, M, (img_size[1], img_size[0]), flags=cv2.INTER_LINEAR)
    cv2.fillPoly(warped, np.int_([pt1]), (0,255))
    cv2.fillPoly(warped, np.int_([pt2]), (0,255))
    return warped, M, Minv
    
    
# base_img = np.copy(Final_Grad_Image)
# warped_img, M, Minv = warping(base_img)
# # plot(base_img, warped_img, 'Original', 'Warped', 'Warped_Image')

In [10]:
## Finding Lines ##
def DrawHisto(img):
    histo = np.sum(img[img.shape[0]//2:,:], axis=0)
#     f = plt.figure(figsize=(10, 6))
#     plt.plot(histo)
#     plt.show()
#     f.savefig('output_images/Histogram.png') #save last pic for report
    return histo

def DrawSlidingWindow(warped_img, histogram):
    out_img = np.dstack((warped_img, warped_img, warped_img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(warped_img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped_img.shape[0] - (window+1)*window_height
        win_y_high = warped_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, warped_img.shape[0]-1, warped_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
  
    return left_fitx, right_fitx, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, ploty, margin

def Visualization(warped_img, left_fitx, right_fitx, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, ploty, margin):
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((warped_img, warped_img, warped_img))*255
    window_img = np.zeros_like(out_img)
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    return out_img, result

# histogram = DrawHisto(warped_img)

# left_fitx, right_fitx, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, ploty, margin = DrawSlidingWindow(warped_img, histogram)
# RB_lane_lines, vis_lane_lines = Visualization(warped_img, left_fitx, right_fitx, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, ploty, margin)

# f = plt.figure(figsize=(10,6))
# plt.imshow(vis_lane_lines)
# plt.plot(left_fitx, ploty, color='yellow')
# plt.plot(right_fitx, ploty, color='yellow')
# plt.xlim(0, 1280)
# plt.ylim(720, 0)
# plt.show()
# # f.savefig('output_images/Sliding_Window_Visualization.png') #save last pic for report

In [11]:
## Finding Radius of Curvature ##

def FindROC(left_fitx, right_fitx, ploty):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/600 # meters per pixel in x dimension

    y_eval = np.max(ploty)

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)

    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
      
    xMax = img.shape[1]*xm_per_pix
    yMax = img.shape[0]*ym_per_pix
    vehicleCenter = xMax / 2
    lineLeft = left_fit_cr[0]*yMax**2 + left_fit_cr[1]*yMax + left_fit_cr[2]
    lineRight = right_fit_cr[0]*yMax**2 + right_fit_cr[1]*yMax + right_fit_cr[2]
    lineMiddle = lineLeft + (lineRight - lineLeft)/2
    diffFromVehicle = vehicleCenter - lineMiddle
    
    return left_curverad, right_curverad, left_fit_cr, right_fit_cr, diffFromVehicle

# left_curverad, right_curverad, left_fit_cr, right_fit_cr, diffFromVehicle = FindROC(left_fitx, right_fitx, ploty)

# print('Left Lane ROC: ', left_curverad/1000, 'km')
# print('Right Lane ROC: ', right_curverad/1000, 'km')
# print('Vehicle Deviation from center: ', diffFromVehicle, 'm')

In [12]:
## Unwarping The Lanes & Lane Area ##

def DisplayPanel(undst, RB_lane_lines, vis_lane_lines, left_fitx, right_fitx, ploty, curverad, diffFromVehicle, Minv):
    # Initialization
    lane_area = np.zeros_like(RB_lane_lines)

    # Collecting left and Right points to find Lane Area
    left_inner_pts = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_inner_pts = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((left_inner_pts, right_inner_pts))

    cv2.fillPoly(lane_area, np.int_([pts]), (0,255,0))

    # Unwarping Colored Lane Lines and Lane Area
    unwarped_lane_area = cv2.warpPerspective(lane_area, Minv, (undst.shape[1], undst.shape[0]), flags=cv2.INTER_LINEAR)
    unwarped_lane_lines = cv2.warpPerspective(RB_lane_lines, Minv, (undst.shape[1], undst.shape[0]), flags=cv2.INTER_LINEAR)
    lane_lines = cv2.addWeighted(unwarped_lane_lines,1,unwarped_lane_area,0.3,0)

    # Final Output Panel Image Generation
    final_output = cv2.add(undst, lane_lines)
    font = cv2.FONT_HERSHEY_SIMPLEX
    text1 = ('Radius of Curvature : {:.4f} km'.format(curverad/1000))            
    if diffFromVehicle > 0:
        text2 = '{:.2f} m right'.format(diffFromVehicle)
    else:
        text2 = '{:.2f} m left'.format(-diffFromVehicle)
    cv2.putText(final_output, text1, (50, 50), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(final_output, 'Vehicle is {} of center'.format(text2), (50, 100), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    white = [255,255,255]     #---Color of the border---
    undst = cv2.copyMakeBorder(undst,5,5,5,5,cv2.BORDER_CONSTANT,value=white)
    final_output = cv2.copyMakeBorder(final_output,5,5,5,5,cv2.BORDER_CONSTANT,value=white)
    vis_lane_lines = cv2.copyMakeBorder(vis_lane_lines,5,5,5,5,cv2.BORDER_CONSTANT,value=white)
    lane_lines = cv2.copyMakeBorder(lane_lines,5,5,5,5,cv2.BORDER_CONSTANT,value=white)
    
    
    horizontal1 = np.hstack((undst, final_output))
    horizontal2 = np.hstack((vis_lane_lines, lane_lines))
    Panel = np.vstack((horizontal1, horizontal2))
    return Panel

# Panel = DisplayPanel(undst, RB_lane_lines, vis_lane_lines, left_fitx, right_fitx, ploty, left_curverad, diffFromVehicle, Minv)

# f = plt.figure(figsize=(20, 20))
# plt.imshow(Panel)
# plt.show()
# # f.savefig('output_images/Panel_Output.png') #save last pic for report

In [13]:
##############################################################################################################################

In [14]:
###########################################
#### PIPELINE FOR ADVANCE LANE FINDING ####
###########################################

In [15]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [16]:
def meanLaneWidth(left_fitx, right_fitx, ploty):
    # calcualtes averae horizontal distance
    # between left and right lane lines
    dataLength = ploty.size
    intervalSize = dataLength//50
    laneWidth = []
    for i in range(0,dataLength, intervalSize):
        dist = right_fitx[i] - left_fitx[i]
        laneWidth.append(dist)
    
    laneWidth = np.array(laneWidth)
    meanLaneWidth = np.mean(laneWidth)
    stddevLaneWidth = np.std(laneWidth)
    
    return meanLaneWidth, stddevLaneWidth

class Line():
    def __init__(self):
        self.left_fitx = None
        self.right_fitx = None
        self.left_fit_cr = None
        self.right_fit_cr = None
        self.radius_of_curvature = None        
        self.line_base_pos = None
        self.left_curverad = None
        self.right_curverad = None
        self.detected = False
        self.dominantLine = False
        
leftLane = Line()
rightLane = Line()

In [17]:
def ALF_Pipeline(img):
    ksize = 15
    lwMeandev = 0.1 # percent
    lwStddev = 5 # pixels
    
    # Gradient & Color Threshold Step
    undst = cv2.undistort(img, mtx, dist, None, mtx)
    gradx = abs_sobel_thresh(undst, orient='x', thresh=(20,255))
    grady = abs_sobel_thresh(undst, orient='y', thresh=(40,255))
    mag_binary = mag_threshold(undst, sobel_kernel=ksize, mag_thresh=(60,255))
    dir_binary = dir_threshold(undst, sobel_kernel=ksize, dir_thresh=(0.5,1.3))
    combined_gradient = np.zeros_like(dir_binary)
    combined_gradient[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    SL_Filter = hls_select(undst, threshS=(160,255))
    B_Filter = lab_bthresh(undst, thresh=(0,100))
    color_filters = np.zeros_like(SL_Filter)
    color_filters[(SL_Filter == 1) | (B_Filter == 1)] = 1
    Final_Grad_Image = np.zeros_like(color_filters)
    Final_Grad_Image[(color_filters == 1) | (combined_gradient == 1)] = 1
    
    # Warping Image
    warped_img, M, Minv = warping(Final_Grad_Image)
    if ((leftLane.detected == False) or (rightLane.detected == False)): 

        histogram = DrawHisto(warped_img)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        # choose the line with more data points for Radius of Curv. display
        if histogram[leftx_base] >= histogram[rightx_base]:
            leftLane.dominantLine = True
        else:
            leftLane.dominantLine = False
    
    left_fitx, right_fitx, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, ploty, margin = DrawSlidingWindow(warped_img, histogram)
    mean, stddev = meanLaneWidth(left_fitx, right_fitx, ploty)
    left_curverad, right_curverad, left_fit_cr, right_fit_cr, diffFromVehicle = FindROC(left_fitx, right_fitx, ploty)  
    
    if (leftLane.dominantLine == True):
            curverad = left_curverad
    else:
            curverad = right_curverad
            
    if ((abs(mean - 600) < lwMeandev*600)  and (stddev < lwStddev)):
        left_fitx = leftLane.left_fitx
        left_fit_cr = leftLane.left_fit_cr
        right_fitx = rightLane.right_fitx
        right_fit_cr = rightLane.right_fit_cr
    else:
        leftLane.left_fitx = left_fitx
        leftLane.left_fit_cr = left_fit_cr
        rightLane.right_fitx = right_fitx
        rightLane.right_fit_cr = right_fit_cr

    
    RB_lane_lines, vis_lane_lines = Visualization(warped_img, left_fitx, right_fitx, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, ploty, margin)
    Panel = DisplayPanel(undst, RB_lane_lines, vis_lane_lines, left_fitx, right_fitx, ploty, curverad, diffFromVehicle, Minv)
    return Panel


In [19]:
white_output = './output_videos/project_video_with_lanes_2.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("./project_video.mp4")
white_clip = clip1.fl_image(ALF_Pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ./output_videos/project_video_with_lanes_2.mp4
[MoviePy] Writing video ./output_videos/project_video_with_lanes_2.mp4


100%|█████████████████████████████████████▉| 1260/1261 [07:42<00:00,  2.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/project_video_with_lanes_2.mp4 

Wall time: 7min 44s
